### 黄牛检测

In [101]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import random
import torch
from torch import nn
from sklearn.model_selection import train_test_split

In [61]:
# 读取数据
file_path = './data/raw_data.xlsx'
sheet_name = 'Sheet1'
data = pd.read_excel(file_path, sheet_name=sheet_name)

In [62]:
# 时间预处理
data['订单创建时间'] = pd.to_datetime(data['订单创建时间'])
data['就诊日期'] = pd.to_datetime(data['就诊日期'])

In [ ]:
# 写入到指定log文件中(LogBERT相关,已废弃)
def write_log(log_file_path):
    # 将数据逐行写入.log 文件
    with open(log_file_path, 'w', encoding='utf-8') as log_file:
        for index, row in data.iterrows():
            # 将每一行转换为字符串并写入日志文件
            row = row.to_dict()
            log_file.write(row['患者ID'] + " : ")
            for k, v in row.items():
                if k != "患者ID":
                    log_file.write(str(v) + " ")
            log_file.write("\n")
    print(f"Data has been written to {log_file_path}")

#### 1. 基于规则的检测

1. IP重复，来自同一个IP，且为超过3个人挂号
2. 用户重复，来自同一个用户，且挂号了超过3个科室/超过两个app_id
3. 时间过早，每天5:00-5:01进行操作的

In [65]:
# 重复值筛选，找出data的seg_name字段中重复数大于limit的行, sort_add和asc_add是检测完之后添加的排序要求
def duplicate_detect(data, seg_name, limit, up=True, sort_add=[], asc_add=[]):
    assert len(sort_add) == len(asc_add)
    value_counts = data[seg_name].value_counts()
    if up:
        # 向上筛
        dup_row = data[data[seg_name].isin(value_counts[value_counts > limit].index)]
    else:
        # 向下筛
        dup_row = data[data[seg_name].isin(value_counts[value_counts < limit].index)]
    sort_by = [seg_name] + sort_add
    asc = [True] + asc_add
    dup_row = dup_row.sort_values(by=sort_by, ascending=asc)
    return dup_row

In [ ]:
# 筛选出同一个dup_seg中有超过limit个unique_seg的ID字段
def unique_dup_filter(data, dup_seg, unique_seg, limit, up=True, get_row=False):
    # 确定是黄牛的行编号
    selects = []
    # IP重复检测
    dup = duplicate_detect(data, dup_seg, limit, up=up, sort_add=[unique_seg], asc_add=[True])
    print("init dup_num:", len(dup))
    if len(dup) == 0:
        return selects
    pos = 0          # 这个IP的起点
    count = 1      # 涉及多少个用户ID
    last = dup[unique_seg].iloc[0] # 上一个用户的ID
    dup_num = len(dup)
    for i in tqdm(range(1, dup_num)):
        if dup[dup_seg].iloc[i] == dup[dup_seg].iloc[pos]:
            if dup[unique_seg].iloc[i] != last:
                # 相同IP下一个新的患者
                count += 1
                last = dup[unique_seg].iloc[i]
        if dup[dup_seg].iloc[i] != dup[dup_seg].iloc[pos] or i == dup_num - 1:
            # 开始检测下一个IP
            if count > limit:
                # 达到重复人数条件
                for j in range(pos, i):
                    selects.append(dup['ID'].iloc[j])
            # 重置
            pos = i
            count = 1
            last = dup[unique_seg].iloc[i]
    print("filtered dup_num:", len(selects))
    if not get_row:
        selects.sort()
        return selects
    else:
        dup_rows = data[data['ID'].isin(selects)]
        dup_rows = dup_rows.sort_values(by=dup_seg, ascending=True)
        return dup_rows

In [ ]:
# 对unique_dup_filter, 在一定范围内遍历limit(基本不用)
# 使用: limits, select_list = grid_traverse(data, '患者ID', '就诊科室名称', 5, 10)
def grid_traverse(data, dup_seg, unique_seg, start, end, gap=1):
    limits = []
    selects_list = []
    for limit in range(start, end, gap):
        print(f"limit: {limit}")
        selects = unique_dup_filter(data, dup_seg, unique_seg, limit)
        limits.append(limit)
        selects_list.append(selects)
    return limits, selects_list

In [69]:
# 时间段过滤器，过滤出每天一段时间内的数据
# 形如daily_filter(data, '5:00:00', '5:01:00')
def daily_filter(data, start, end):
    return data[(data['订单创建时间'].dt.time >= pd.to_datetime(start).time()) &
                    (data['订单创建时间'].dt.time <= pd.to_datetime(end).time())]

In [ ]:
# 时间分段统计(数据分析而非处理, 只会打印出每个小时内各个操作的次数)
def hour_count():
    print("\t\t总数\t已挂号  医保换号  已退号  窗口退号  无号退款  超时取消")
    for i in range(5, 23):
        time_filter = data[(data['订单创建时间'].dt.time >= pd.to_datetime(f'{i}:00:00').time()) &
                     (data['订单创建时间'].dt.time <= pd.to_datetime(f'{i+1}:00:00').time())]
        counts = time_filter['状态'].value_counts()
        print(f"{i}:00 - {i+1}:00 \t{len(time_filter)}\t{counts['已挂号']}\t{counts['医保换号']}\t {counts['已退号']}\t"
              f"   {counts['窗口退号']}\t   {counts['无号退款']}\t    {counts['超时取消']}\t")

# 一个小时内按分钟打印操作次数(数据分析而非处理)
def minute_line(data, hour):
    count = []
    for i in tqdm(range(0, 59, 5)):
        start = str(i)
        end = str(i+5)
        if len(start) < 2:
            start = '0' + start
        if len(end) < 2:
            end = '0' + end 
        time_filter = data[(data['订单创建时间'].dt.time >= pd.to_datetime(f'{hour}:{start}:00').time()) &
                     (data['订单创建时间'].dt.time <= pd.to_datetime(f'{hour}:{end}:00').time())]
        count.append(len(time_filter))
    plt.plot(count)

In [ ]:
# 恰好在16:00进行第二天/下一周操作(因为这里面很有可能有很多正常人,不建议使用)
def hurry_sixteen(data, get_row=False):
    gap = daily_filter(data, '16:00:00', '16:00:01')
    time_diff = (gap['就诊日期'] - gap['订单创建时间']).dt.days
    hurry_row = gap[(time_diff == 0) | (time_diff == 6)]
    if get_row:
        return hurry_row
    else:
        return hurry_row['ID'].tolist()

In [ ]:
# 将之前一次答案中的数据提取成列表(需要配套data文件夹中的result1/2/3...使用)
def get_list(res_id, file_path=""):
    if file_path == "":
        file_path = f'./data/result{res_id}.txt'
    with open(file_path, 'r') as file:
        lis = [int(line.strip()) for line in file]
    return lis

In [ ]:
# 两个list进行对比(相比增加的,减少的,相同的)
def lis_cmp(lis1, lis2, ret=False, show=False):
    set1 = set(lis1)
    set2 = set(lis2)
    new_ele = set2 - set1
    miss_ele = set1 - set2
    same_ele = set1.intersection(set2)
    if show:
        print(f"lis1: {len(lis1)},\tlis2: {len(lis2)},\tmore: {len(new_ele)},"
            f"\tmiss: {len(miss_ele)},\tsame = {len(same_ele)}")
    if ret:
        return list(new_ele), list(miss_ele), list(same_ele)

# 与之前一次答案进行对比
def res_cmp(res_id, lis, ret=False, show=False):
    lis1 = get_list(res_id)
    res = lis_cmp(lis1, lis, ret, show)
    if ret:
        return res

In [ ]:
# 将一个list写作答案(写到data/result.txt中,remove_white选项是是否去除白名单)
def write_list(lis, remove_white=True):
    if remove_white:
        white_lis = get_list(0, './data/white.txt')
        lis = list(set(lis) - set(white_lis))
    # 打开一个文件进行写入，如果文件不存在则创建
    with open('./data/result.txt', 'w', encoding='utf-8') as file:
        # 遍历列表中的每个元素
        for item in lis:
            # 将每个元素写入文件，每个元素后面加上换行符
            file.write(str(item) + '\n')
        print(f"Total line: {len(lis)}")

In [ ]:
# data中退号超过limit词的(这条规则效果似乎不算太好)
def frequent_drop(data, limit, get_row=False):
    drop = data[data['状态'] == '已退号']
    mass_drop = duplicate_detect(drop, "患者ID", limit)
    if get_row:
        return mass_drop['ID'].tolist()
    else:
        return mass_drop

In [ ]:
# 简单筛选不是黄牛的用户(效果不确定)
def normal_user(data):
    # 黑名单
    ip_black = duplicate_detect(data, 'IP_ADDRESS', 2)['IP_ADDRESS'].to_list()
    user_black = duplicate_detect(data, '患者ID', 4)['患者ID'].to_list()
    # 删除黑名单中对应的所有ip和患者ID
    white_rows = data[~data['IP_ADDRESS'].isin(ip_black)]
    white_rows = white_rows[~data['患者ID'].isin(user_black)]
    return white_rows
    # 基于用户分组的进一步筛选
    # depart_limit = white_rows.groupby('患者ID').filter(lambda group: group['就诊科室名称'].nunique() <= 2)
    # app_limit = depart_limit.groupby('患者ID').filter(lambda group: group['APPID'].nunique() <= 2)
    # area_limit = app_limit[(app_limit['省份'] == '北京') | (app_limit['省份'] == '河北')]
    # time_limit = area_limit[((area_limit['订单创建时间'].dt.time >= pd.to_datetime(f'6:00:00').time()) &
    #                  (area_limit['订单创建时间'].dt.time <= pd.to_datetime(f'16:00:00').time())) | 
    #                  (area_limit['订单创建时间'].dt.time >= pd.to_datetime(f'17:00:00').time())]
    # return time_limit

In [76]:
# 根据权重在多个不同的list中进行合并筛选
def weighted_selection(lists, weights, n, limit=0):
    # 创建一个字典来存储元素的总权重
    total_weights = defaultdict(float)

    # 遍历每个列表及其对应的权重
    for lst, weight in zip(lists, weights):
        for element in lst:
            total_weights[element] += weight  # 累加权重

    # 将字典转换为列表，并筛选出权重大于 limit 的元素
    filtered_elements = {k: v for k, v in total_weights.items() if v >= limit}

    # 按照权重排序
    sorted_elements = sorted(filtered_elements.items(), key=lambda x: x[1], reverse=True)

    # 选择前 n 个元素及其权重
    top_n_elements = sorted_elements[:n]

    return top_n_elements

#### 2. 基于聚类的分析

In [77]:
def ip_encoder(ip):
    parts = list(map(int, ip.split('.')))
    return (parts[0] << 24) + (parts[1] << 16) + (parts[2] << 8) + parts[3]

#### 3. 基于学习

In [78]:
state2num = {
    '已退号': 0,
    '已挂号': 1,
    '窗口退号': 2,
    '医保换号': 3,
    '无号退款': 4,
    '超时取消': 5,
    '主动取消': 6,
    np.nan: 7
}

def get_embed_list(state_list_pd, seg_name, series_name):
    state_lists = state_list_pd.groupby(seg_name)[series_name].apply(list).reset_index()
    state_lists = state_lists[series_name].to_list()
    for i in range(len(state_lists)):
        state_lists[i] = [str(state2num[str_]) for str_ in state_lists[i]]
    return state_lists

def generate_train_test(data, seg_name, series_name, normal_func, limit=2000, path='../LogBERT/output/rd/'):
    # train
    normal_rows = normal_func(data)
    normal_id = normal_rows[seg_name].to_list()
    sample_id = random.sample(normal_id, limit)
    normals = data[data[seg_name].isin(sample_id)]
    state_lists = get_embed_list(normals, seg_name, series_name)
    with open(path + "train", 'w') as train_file:
        for states in state_lists:
            train_file.write(" ".join(states) + "\n")

    # test_normal
    unsample_id = list(set(normal_id) - set(sample_id))
    test_normals = data[data[seg_name].isin(unsample_id)]
    state_lists = get_embed_list(test_normals, seg_name, series_name)
    with open(path + "test_normal", 'w') as train_file:
        for states in state_lists:
            train_file.write(" ".join(states) + "\n")

    # test_abnormal
    res_id = get_list(5)
    test_abnormals = data[data['ID'].isin(res_id)]
    state_lists = get_embed_list(test_abnormals, seg_name, series_name)
    with open(path + "test_abnormal", 'w') as train_file:
        for states in state_lists:
            train_file.write(" ".join(states) + "\n")

##### LogBERT 魔改记录

1. logbert.py里的options["min_len"]改成了1, epoch也改成了2
2. sample.py中line = line.squeeze()改为line.ravel()
3. sample.py中logkey_seq_pairs = np.array(logkey_seq_pairs,)加上了dtype=object，predict_log.py中如果报错也添加这个就行
4. train_log.py中epoch > 0及生成center

In [ ]:
# 将一个list写入白名单
def add_white(lis, path='./data/white.txt'):
    former_list = []
    if os.path.isfile(path):
        former_list = get_list(0, path)
    lis = list(set(lis + former_list))
    with open(path, 'w', encoding='utf-8') as file:
        for item in lis:
            file.write(str(item) + '\n')
        print(f"Total line: {len(lis)}")

In [113]:
lis = get_list(8)
for i in range(1, 8):
    res_cmp(i, lis, show=True)

lis1: 6339,	lis2: 9599,	more: 4881,	miss: 1621,	same = 4718
lis1: 8480,	lis2: 9599,	more: 4556,	miss: 3437,	same = 5043
lis1: 5824,	lis2: 9599,	more: 5712,	miss: 1937,	same = 3887
lis1: 8921,	lis2: 9599,	more: 4145,	miss: 3467,	same = 5454
lis1: 3210,	lis2: 9599,	more: 6728,	miss: 339,	same = 2871
lis1: 8352,	lis2: 9599,	more: 2706,	miss: 1459,	same = 6893
lis1: 9696,	lis2: 9599,	more: 2521,	miss: 2618,	same = 7078


#### 工作区

In [143]:
# 省份, 患者ID, 就诊科室名称, 医生姓名, 状态

# 按照用户分组提取出操作的时间序列，然后嵌入成一个torch.tensor
def feature_extract(rows, prov_embedder, depart_embedder, doc_embedder, state_embedder):
    res_list = []
    # 根据User分成dataFrame的list
    user_group = rows.groupby('患者ID')
    data_list = [group.reset_index(drop=True) for name, group in user_group]
    
    # 每一组根据时间进行排序
    for i in range(len(data_list)):
        data_list[i] = data_list[i].sort_values(by='就诊日期', ascending=True)
        
        prov_tensor = torch.tensor(rows['prov_id'].values)
        depart_tensor = torch.tensor(rows['depart_id'].values)
        doc_tensor = torch.tensor(rows['doc_id'].values)
        state_tensor = torch.tensor(rows['state_id'].values)

        prov_tensor = prov_embedder(prov_tensor)
        depart_tensor = depart_embedder(depart_tensor)
        doc_tensor = doc_embedder(doc_tensor)
        state_tensor = state_embedder(state_tensor)

        embedding = torch.cat((prov_tensor, depart_tensor, doc_tensor, state_tensor), dim=1)

        res_list.append(embedding)
        break
    
    return res_list

In [126]:
# 数据预处理
# 先对类别列进行编码
data['prov_id'] = pd.factorize(data['省份'])[0]
data['depart_id'] = pd.factorize(data['就诊科室名称'])[0]
data['doc_id'] = pd.factorize(data['医生姓名'])[0]
data['state_id'] = pd.factorize(data['状态'])[0]

In [127]:
# 准备数据
## 黄牛
abnormal_id = get_list(6)
abnormal_rows = data[data['ID'].isin(abnormal_id)]
## 正常人

In [146]:
# 特征提取
# 创建embedding
PROV_DIM = 1
DEPART_DIM = 2
DOC_DIM = 1
STATE_DIM = 2
prov_embbeder = nn.Embedding(max(data['prov_id']) + 1, PROV_DIM)
depart_embbeder = nn.Embedding(max(data['depart_id']) + 1, DEPART_DIM)
doc_embbeder = nn.Embedding(max(data['doc_id']) + 1, DOC_DIM)
state_embbeder = nn.Embedding(max(data['state_id']) + 1, STATE_DIM)

In [147]:
feature_extract(abnormal_rows, prov_embbeder, depart_embbeder, doc_embbeder, state_embbeder)

IndexError: index out of range in self

In [81]:
# 模型定义
class TransLog(nn.Module):
    def __init__(self, input_dim, model_dim, n_heads, num_classes, num_layers, dropout=0.1):
        super(TransLog, self).__init__()
        
        # 输入嵌入层
        self.embedding = nn.Linear(input_dim, model_dim)
        
        # Dropout层
        self.dropout = nn.Dropout(dropout)

        # Transformer 编码器
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=n_heads, dropout=dropout),
            num_layers=num_layers
        )
        
        # 分类层
        self.fc = nn.Linear(model_dim, num_classes)
        

    def forward(self, x):
        # 输入形状: (seq_len, batch_size, input_dim)
        x = self.embedding(x)  # (seq_len, batch_size, model_dim)
        x = self.dropout(x)
        
        # 通过 Transformer 编码器
        x = self.transformer_encoder(x)  # (seq_len, batch_size, model_dim)
        
        # 取最后一个时间步的输出
        x = x[-1, :, :]  # (batch_size, model_dim)
        
        # 分类
        x = self.fc(x)  # (batch_size, num_classes)
        
        return x

# 超参数设置
input_dim = 10  # 输入特征的维度（例如，患者的特征数量）
model_dim = 64  # Transformer 模型的维度
n_heads = 8     # 注意力头的数量
num_classes = 2 # 分类数量（例如，黄牛与非黄牛）
num_layers = 4  # Transformer 层数
dropout = 0.1   # Dropout 比例

# 创建模型
model = TransLog(input_dim, model_dim, n_heads, num_classes, num_layers, dropout)

# 示例输入
seq_len = 5    # 序列长度
batch_size = 3 # 批次大小
example_input = torch.rand(seq_len, batch_size, input_dim)  # 随机生成示例输入

# 前向传播
output = model(example_input)

print("模型输出形状:", output.shape)  # 应该是 (batch_size, num_classes)

模型输出形状: torch.Size([3, 2])


c:\Library\Python\anaconda\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [82]:
# # 数据准备
# # 假设您有以下数据
# # features: (num_samples, seq_len, input_dim)
# # labels: (num_samples)

# # 生成示例数据
# num_samples = 1000
# seq_len = 5
# input_dim = 10
# num_classes = 2

# features = torch.rand(num_samples, seq_len, input_dim)  # 随机生成特征
# labels = torch.randint(0, num_classes, (num_samples,))   # 随机生成标签

# # 切分数据集
# X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

In [83]:
# # 模型训练
# # 创建模型
# model = TransLog(input_dim, model_dim, n_heads, num_classes, num_layers, dropout)

# # 定义损失函数和优化器
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # 训练参数
# num_epochs = 20
# batch_size = 32

# # 训练循环
# for epoch in range(num_epochs):
#     model.train()  # 设置模型为训练模式
#     for i in range(0, len(X_train), batch_size):
#         # 获取当前批次的输入和标签
#         inputs = X_train[i:i + batch_size]
#         labels = y_train[i:i + batch_size]

#         optimizer.zero_grad()  # 清空梯度
#         outputs = model(inputs)  # 前向传播
#         loss = criterion(outputs, labels)  # 计算损失
#         loss.backward()  # 反向传播
#         optimizer.step()  # 更新参数

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [84]:
# # 模型评估
# model.eval()  # 设置模型为评估模式
# with torch.no_grad():  # 禁用梯度计算
#     val_outputs = model(X_val)
#     _, predicted = torch.max(val_outputs, 1)  # 获取预测类别
#     accuracy = (predicted == y_val).sum().item() / len(y_val) * 100  # 计算准确率

# print(f'Validation Accuracy: {accuracy:.2f}%')

In [85]:
# # 模型预测
# def predict(model, input_sequence):
#     model.eval()  # 设置模型为评估模式
#     with torch.no_grad():  # 禁用梯度计算
#         output = model(input_sequence)  # 前向传播
#         _, predicted = torch.max(output, 1)  # 获取预测类别
#     return predicted

# # 示例输入
# new_patient_sequence = torch.rand(1, seq_len, input_dim)  # 随机生成一个新患者的序列
# prediction = predict(model, new_patient_sequence)

# print(f'预测类别: {prediction.item()}')  # 输出预测结果

#### 流放地

In [86]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.cluster import KMeans
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import MinMaxScaler
# # 特征选择
# used_data = pd.DataFrame()
# used_data['ID'] = data['ID']
# used_data['start_t'] = data['订单创建时间'].dt.hour * 3600 + data['订单创建时间'].dt.minute * 60 + data['订单创建时间'].dt.second
# used_data['delta_t'] = (data['就诊日期'] - data['订单创建时间']).astype('int64') / 10**9
# used_data['pid'] = data['患者ID']

# # 统计每个患者ID对应的不同APPID数量
# app_num = data.groupby('患者ID')['APPID'].nunique().reset_index()
# app_num.columns = ['pid', 'app_num']  # 重命名列
# used_data = used_data.merge(app_num, on='pid', how='left')

# # 每个患者的科室挂号数
# dorm_num = data.groupby('患者ID')['就诊科室名称'].nunique().reset_index()
# dorm_num.columns = ['pid', 'dorm_num']  # 重命名列
# used_data = used_data.merge(dorm_num, on='pid', how='left')

# used_data['province'] = data['省份']
# used_data['ip'] = data['IP_ADDRESS'].apply(ip_encoder)
# used_data['status'] = data['状态']

# features = ['start_t', 'delta_t', 'pid', 'province', 'ip', 'status', 'app_num']

# # 归一化
# scaler = MinMaxScaler()  # 或使用 StandardScaler()
# used_data[['start_t', 'delta_t', 'ip']] = scaler.fit_transform(used_data[['start_t', 'delta_t', 'ip']])

# # 使用 ColumnTransformer 进行特征处理
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', SimpleImputer(strategy='constant'), ['delta_t', 'start_t', 'ip', 'app_num']),
#         ('cat', OneHotEncoder(), ['pid', 'province', 'status'])  # 分类特征处理
#     ],
#     remainder='drop'
# )
# # 创建聚类管道
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('kmeans', KMeans(n_clusters=2, random_state=0))
# ])

# # 拟合模型
# pipeline.fit(used_data[features])

# # 获取聚类标签
# used_data[f'cluster'] = pipeline.predict(used_data[features])